In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from collections import Counter
from subprocess import check_output

In [2]:
path_to_data = "/media/raph/Elements/ml1/churn/"
transactions_chunk_size = 10000
nrows_for_members = None
nrows_for_train = None
nrows_for_train_v2 = None
nrows_for_test = None
nrows_for_test_v2 = None

In [3]:
train = pd.read_csv(path_to_data+"train_v2.csv", nrows=nrows_for_train_v2)
train = pd.concat((train, pd.read_csv(path_to_data+"train.csv", nrows=nrows_for_train)), axis=0, ignore_index=True).reset_index(drop=True)
members = pd.read_csv(path_to_data + "members_v3.csv", nrows=nrows_for_members)
test = pd.read_csv(path_to_data+"sample_submission_v2.csv", nrows=nrows_for_test_v2)
test = pd.concat((test, pd.read_csv(path_to_data+"sample_submission_zero.csv", nrows=nrows_for_test)), axis=0, ignore_index=True).reset_index(drop=True)

In [4]:
training = pd.merge(left=train, right=members, how='left', on=['msno'])
del train
testing = pd.merge(left=test, right=members, how='left', on=['msno'])
del test
del members

In [5]:
# changing type to int and putting -1 for missing values
training['city'] = training.city.apply(lambda x: int(x) if pd.notnull(x) else -1)
training['registered_via'] = training.registered_via.apply(lambda x: int(x) if pd.notnull(x) else -1)
training['bd'] = training.bd.apply(lambda x: int(x) if pd.notnull(x) else -1)
training['bd'] = training.bd.apply(lambda x: x if (10<x<100) else -1)

# encode gender
genders_encoding = {'male': 0, 'female': 1}
training['gender'] = training.gender.apply(lambda x: genders_encoding[x] if pd.notnull(x) else -1)

# changing date formats
training['registration_init_time'] = training.registration_init_time.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN")

training.set_index('msno', inplace=True)

In [6]:
# reducing memory usage:
from tools import change_datatype, change_datatype_float
change_datatype(training)
change_datatype_float(training)

change_datatype(testing)
change_datatype_float(testing)

In [7]:
def reformat_transactions(df):
    df['transaction_date'] = df.transaction_date.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN")
    df['membership_expire_date'] = df.membership_expire_date.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN")
    df['payment_method_id'] = df.payment_method_id.apply(lambda x: int(x) if pd.notnull(x) else -1)
    boolean_indexes = df["transaction_date"] < datetime.strptime("20170201", "%Y%m%d").date()
    indexes_to_drop = [e[0] if e[1] else None for e in boolean_indexes.iteritems()]
    indexes_to_keep = set(range(df.shape[0])) - set(indexes_to_drop)
    df = df.take(list(indexes_to_keep))

In [8]:
df_iter = pd.read_csv(path_to_data+'transactions.csv', low_memory=False, iterator=True, chunksize=transactions_chunk_size)

training["total_number_of_transactions"] = 0

training["current_number_of_transactions"] = 0
training.drop(['current_number_of_transactions'], axis=1, inplace=True)

In [9]:
i = 0
print("starting iteration...")
for transactions in df_iter:
    print("i="+str(i))
    reformat_transactions(transactions)
    user_count = Counter(transactions['msno']).most_common()
    user_count = pd.DataFrame(user_count)
    user_count.columns = ['msno', 'current_number_of_transactions']
    user_count.set_index('msno', inplace=True)
    training = pd.merge(left=training, right = user_count, how = 'left', left_index=True, right_index=True)
    training['current_number_of_transactions'] = training.current_number_of_transactions.apply(lambda x: int(x) if pd.notnull(x) else 0)
    training["total_number_of_transactions"] += training["current_number_of_transactions"]
    training.drop(['current_number_of_transactions'], axis=1, inplace=True)
    i+=1
    
    if i >0:
        break

starting iteration...
i=0


In [10]:
training.head()

,is_churn,city,bd,gender,registered_via,registration_init_time,total_number_of_transactions
msno,,,,,,,
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,1,-1,-1,7,2014-06-08,0
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,1,-1,-1,7,2014-06-08,0
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,15,26,0,9,2010-11-18,0
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,15,26,0,9,2010-11-18,0
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0,1,-1,-1,7,2014-09-27,0


In [11]:
i = 0
training.reset_index(inplace=True)
training_copy = training.copy()
training_copy.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,total_number_of_transactions
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,1,-1,-1,7,2014-06-08,0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,1,-1,-1,7,2014-06-08,0
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,15,26,0,9,2010-11-18,0
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,15,26,0,9,2010-11-18,0
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0,1,-1,-1,7,2014-09-27,0


In [12]:
print("starting iteration, looking for most recent transaction...")
for transactions in df_iter:
    print("i="+str(i))
    
    reformat_transactions(transactions)
    recent_transactions = transactions.sort_values(['transaction_date']).groupby('msno').first()
    recent_transactions.reset_index(inplace=True)
    temp_training = pd.merge(left=training_copy, right=recent_transactions, how = 'right', on=['msno'], right_index=True)
    training = pd.concat((training, temp_training))
    training = training.sort_values(['transaction_date']).groupby('msno').first()
    
    i+=1
    if i >0:
        break

starting iteration, looking for most recent transaction...
i=0


In [13]:
del training_copy
training.head()

,actual_amount_paid,bd,city,gender,is_auto_renew,is_cancel,is_churn,membership_expire_date,payment_method_id,payment_plan_days,plan_list_price,registered_via,registration_init_time,total_number_of_transactions,transaction_date
msno,,,,,,,,,,,,,,,
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,NaN,-1.0,1.0,-1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,7.0,2014-06-08,0.0,NaN
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,NaN,26.0,15.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,9.0,2010-11-18,0.0,NaN
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,NaN,-1.0,1.0,-1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,7.0,2014-09-27,0.0,NaN
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,NaN,21.0,18.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,7.0,2016-03-15,0.0,NaN
++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,NaN,32.0,22.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,9.0,2012-01-22,0.0,NaN
